# Tutorial on Python Exh calculator

## Imports

To complete the tutorial, run the following imports:

In [5]:
from exh import *
from formula import *
from worlds import Universe

## Formulas

### Creating formulas

Formulas are built from logically independent propositional variables. Each variable is associated with a numerical index. Two variables that share the same index will always receive the same truth-value ; two variables with different index are logically independent. You can create variables as follows:

In [7]:
# Creates a variable with index 4
d = Var(3)

By default, **formula.py** creates three independent variables *a*, *b*, and *c* with index 0,1 and 2 respectively. From these variables, one can build standard propositional formulas using the operators &, | and ~.

In [8]:
# f : a and not b
f = a & ~b
# g: (a or b) and (c or d)
g = (a | b) & (c | d)

A formula's make-up can be displayed using Python *print*

In [9]:
print(f)
print(g)

A0 and [not[A1]]
A0 or [A1] and [A2 or [A3]]


### Evaluating formulas

The truth value of a formula against a particular assignment of values to the variables can be computed. Assignments are provided as *NumPy* boolean arrays, where the value at index *i* reflects the value assigned to the variable with index *i*.

In [11]:
# Assigning variables with index 0 and 2 to true and variables with index 1 and 3 to false
assignment = np.array([[True, False, True, False]])

# evaluating f and g against assignment
print("Value of f:",f.evaluate(assignment))
# Output: a and not b = true and not false = true
print("Value of g:",g.evaluate(assignment))
# Output: (a or b) and (c or d) = (true or false) and (true or false) = true
 

Value of f: [ True]
Value of g: [ True]


Truth-values can also be obtained for several assignments at the same time. The same function is used.

In [12]:
assignment = np.array([[True, False, True, False],[True, True, False, False], [True, True, False, True]])
print("Values of f:",f.evaluate(assignment))
print("Values of g:",g.evaluate(assignment))

Value of f: [ True False False]
Value of g: [ True False  True]


One may sometimes want to evaluate a formula against all possible assignments of truth-values. To do that, one construct a *Universe* object. This object constructs all possible assignments of truth-values to variables within a certain range. The range is provided as an argument to the constructor.

In [13]:
# Creating all possible assignments for variables from index 0 to index 3
u = Universe(4)

# Assignments are stored in worlds : we can print them
print(u.worlds)

[[False False False False]
 [ True False False False]
 [False  True False False]
 [ True  True False False]
 [False False  True False]
 [ True False  True False]
 [False  True  True False]
 [ True  True  True False]
 [False False False  True]
 [ True False False  True]
 [False  True False  True]
 [ True  True False  True]
 [False False  True  True]
 [ True False  True  True]
 [False  True  True  True]
 [ True  True  True  True]]


We can then use these assignments to compute the truth-tables of the propositional formulas.

In [14]:
# Getting all possible truth-values for f and g using Formula.evaluate
print("All possible values of f", f.evaluate(u.worlds))
print("All possible values of g", g.evaluate(u.worlds))

All possible values of f [False  True False False False  True False False False  True False False
 False  True False False]
All possible values of g [False False False False False  True  True  True False  True  True  True
 False  True  True  True]


There is an alternative to the code above. The *Universe* object can be used to evaluate formulas as well. The minor advantage is that it can do it in bulks

In [15]:
# Getting all possible truth-values for f and g using Worlds.evaluate
print("All possible values of f", u.evaluate(f))
print("All possible values of g", u.evaluate(g))

# Getting all possible truth-values for f and g in one line
print("All possible values of f", u.evaluate(f, g))

All possible values of f [[False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]
 [False]
 [ True]
 [False]
 [False]]
All possible values of g [[False]
 [False]
 [False]
 [False]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]
 [False]
 [ True]
 [ True]
 [ True]]
All possible values of f [[False False]
 [ True False]
 [False False]
 [False False]
 [False False]
 [ True  True]
 [False  True]
 [False  True]
 [False False]
 [ True  True]
 [False  True]
 [False  True]
 [False False]
 [ True  True]
 [False  True]
 [False  True]]


## Exhaustifying formulas

### Stipulated alternatives

To compute the result of exhaustifying a formula *f* against a given set of alternatives, let's construct the object *Exh*. This object can be constructing by specifying the prejacent and the alternatives.

**Note:** The computation of innocent inclusion/exclusion is done upon creation of the *Exh* object. Expect slow-down at this stage if the formula is complicated.

In [16]:
# Stipulated alternatives
h = Exh(a | b, alts = [a, b, a & b])

[[False  True  True]
 [ True False  True]
 [False False False]]
[[ True False]
 [False  True]]


The program does not give you a representation of what *h* is, but one can confirm that it is the exclusive or meaning by looking at its truth-table, as follows:

In [17]:
print("Truth-table of Exh(a or b)", Universe(2).evaluate(h))

[[False]
 [ True]
 [ True]
 [False]]


One may want further diagnostics from the computation. The following code explains how one displays the maximal sets generated for inclusion and exclusion and the corresponding innocently excludable and innocently includable alternatives

In [18]:
# Exclusion : displaying maximal sets and innocently excludable alternatives
print("Maximal exclusion sets",h.e.maximalExclSets)
print("IE alternatives",h.e.innocently_excl)

Maximal exclusion sets [[A1, A0 and [A1]], [A0, A0 and [A1]]]
IE alternatives [A0 and [A1]]


In [19]:
# Inclusion : displaying maximal sets and innocently includable alternatives
print("Maximal inclusion sets",h.e.maximalInclSets)
print("II alternatives",h.e.innocently_incl)

Maximal exclusion sets [[A0], [A1]]
IE alternatives []


The program can fairly quickly compute the result of exhaust, so long as the number of independent variables stays small (n independent variables -> 2^n instructions). The number of alternatives has little to no influence on the rapidity.

In [22]:
# There are many alternatives here ; see next section for automatic alternative computations
h1 = Exh(a|b|c|d, alts = [a,b,c,d,a&b,a&c,b&c,a&d,b&d,c&d,a&b&c,b&c&d,a&c&d,b&c&d,a&b&c&d])
u = Universe(4)
print("Worlds:")
print(u.worlds)
print("Truth-table of Exh(a or b or c or d)")
print(u.evaluate(h1))


[[False  True  True  True  True  True  True  True  True  True  True  True
   True  True  True]
 [ True False  True  True  True  True  True  True  True  True  True  True
   True  True  True]
 [False False  True  True False  True  True  True  True  True  True  True
   True  True  True]
 [ True  True False  True  True  True  True  True  True  True  True  True
   True  True  True]
 [False  True False  True  True False  True  True  True  True  True  True
   True  True  True]
 [ True False False  True  True  True False  True  True  True  True  True
   True  True  True]
 [False False False  True False False False  True  True  True False  True
   True  True  True]
 [ True  True  True False  True  True  True  True  True  True  True  True
   True  True  True]
 [False  True  True False  True  True  True False  True  True  True  True
   True  True  True]
 [ True False  True False  True  True  True  True False  True  True  True
   True  True  True]
 [False False  True False False  True  True False 

### Automatic alternatives

When not specified, the alternatives to the prejacent are computed in a Katzirian manner: all alternatives are considered that can be obtained from the prejacent by sub-constituent and scalar substitutions. Which alternatives were obtained by this process can be probed after the object is constructed.

In [23]:
h2 = Exh (a | b | c)
print("Computed alternatives", h2.e.alts)

[[ True  True  True  True  True False False  True  True False False  True
   True]
 [ True  True  True  True  True False  True False  True False  True False
   True]
 [ True  True  True  True False False False False False False False False
   True]
 [ True  True  True  True False False False False  True  True  True  True
  False]
 [ True False False  True False False False False  True False False  True
  False]
 [ True False  True False False False False False  True False  True False
  False]
 [False False False False False False False False False False False False
  False]]
[[False  True  True False  True  True False False]
 [False  True False  True  True False  True False]
 [ True  True  True  True False False False  True]]
Computed alternatives [A0 and [A1] and [A2], A0 or [A1] and [A2], A0 and [A2], A1 and [A2], A0 and [A1] or [A2], A0 or [A1] or [A2], A0 or [A2], A1 or [A2], A0 and [A1], A0 or [A1], A0, A1, A2]
